In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests4

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn,return_model=False):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    if return_model:
        return ll1,grad1,hess1,params1,model1_deriv,ll2,grad2,hess2,params2,model2_deriv
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)

(1000, 1) (1, 1)


# Ex 0

In [3]:
beta = 0

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.86   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.14   \\
\hline
\end{tabular}


llr:14.690931031021915, std:24.58700473506957
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.86, 0.  , 0.14]), 14.690931031021915, 24.58700473506957, 21.376547039268093)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.89   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.11   \\
\hline
\end{tabular}


llr:10.49140090522493, std:15.833784524710307
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.89, 0.  , 0.11]), 10.49140090522493, 15.833784524710307, 18.161801840352737)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.91   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.09   \\
\hline
\end{tabular}


llr:13.293625457723289, std:14.979031849961707
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.91, 0.  , 0.09]), 13.293625457723289, 14.979031849961707, 22.089271001614108)


# Ex 0.5

In [7]:
beta = 0.5

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.87 & 1.00 & 0.93   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.13 & 0.13 & 0.00 & 0.07   \\
\hline
\end{tabular}


llr:15.88910930674508, std:44.3938772996187
(array([0.87, 0.  , 0.13]), array([0.87, 0.  , 0.13]), array([1., 0., 0.]), array([0.93, 0.  , 0.07]), 15.88910930674508, 44.3938772996187, 43.77484335412522)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.88 & 0.88 & 1.00 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.12 & 0.12 & 0.00 & 0.03   \\
\hline
\end{tabular}


llr:13.965340742259537, std:53.7396839128556
(array([0.88, 0.  , 0.12]), array([0.88, 0.  , 0.12]), array([1., 0., 0.]), array([0.97, 0.  , 0.03]), 13.965340742259537, 53.7396839128556, 58.69551744108898)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.10 & 0.10 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:9.14333255021056, std:85.54558953250204
(array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([1., 0., 0.]), array([1., 0., 0.]), 9.14333255021056, 85.54558953250204, 79.27972581195046)


# Ex 1

In [11]:
beta = 1

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 1.00 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.00 & 0.01   \\
\hline
\end{tabular}


llr:21.112510587088547, std:89.29096096106893
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), 21.112510587088547, 89.29096096106893, 82.73321951230285)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 1.00 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.00 & 0.01   \\
\hline
\end{tabular}


llr:16.74767264208346, std:121.61824301071586
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), 16.74767264208346, 121.61824301071586, 115.42014606113433)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.04 & 0.04 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:14.055284662422382, std:141.79879903174606
(array([0.96, 0.  , 0.04]), array([0.96, 0.  , 0.04]), array([1., 0., 0.]), array([1., 0., 0.]), 14.055284662422382, 141.79879903174606, 161.78428728066075)


# Ex 1.5

In [23]:
beta = 1.5

In [ ]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

In [ ]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

In [ ]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

# Ex 2

In [19]:
beta = 2

In [20]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 1.00 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.00 & 0.01   \\
\hline
\end{tabular}


llr:-10.46857629317617, std:149.52834272746446
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), -10.46857629317617, 149.52834272746446, 157.43621103021403)


In [21]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 1.00 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:26.896315300525224, std:194.67515911586173
(array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([1., 0., 0.]), array([1., 0., 0.]), 26.896315300525224, 194.67515911586173, 229.89871452963746)


In [22]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 1.00 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.02 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:-9.336227662316158, std:283.47358086881496
(array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([1., 0., 0.]), array([1., 0., 0.]), -9.336227662316158, 283.47358086881496, 320.6895569646673)
